# Transporter analysis of bacillus mother-spore

In [121]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
from collections import defaultdict
import pickle

# third party imports
import pandas as pd
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from cobrame.util.helper_functions import *

import copy
from scipy import stats
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
ecoli_files = dirname(abspath(bacillusme.__file__))
pd.set_option('display.max_colwidth', None)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Predicted production of amino acids

In [2]:
with open("./sporeme_solution_v4_proteomics_spoAQ.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [122]:
pd.DataFrame.from_dict({'flux':sporeme.solution.x_dict}).to_csv('Supplementary File 2.csv')

In [142]:
flux_based_reactions(sporeme,"3mob_s",growth_symbol="sigma",only_types=["MetabolicReaction"])

,lb,ub,rxn_flux,met_flux,reaction
DHAD1_FWD_BSU21870-MONOMER_mod_4fe4s_s,0,999999,0.00221309,0.00221309,23dhmb_s --> 3mob_s + -4.27350427350427e-6*sigma BSU21870-MONOMER_mod_4fe4s_s + h2o_s
VALDHr_REV_BSU24080-MONOMER_s,0,999999,0.00123982,-0.00123982,3mob_s + h_s + nadh_s + nh4_s --> -1.25190676006434e-5*sigma BSU24080-MONOMER_s + h2o_s + nad_s + val__L_s
VALTA_REV_BSU38550-MONOMER_s,0,999999,0.000628319,-0.000628319,3mob_s + glu__L_s --> -4.27350427350427e-6*sigma BSU38550-MONOMER_s + akg_s + val__L_s
OIVD2_FWD_CPLX000-59_s,0,999999,0.000344947,-0.000344947,3mob_s + coa_s + nad_s --> -4.27350427350427e-6*sigma CPLX000-59_s + co2_s + ibcoa_s + nadh_s
VALTA_REV_BSU02390-MONOMER_mod_6:pydx5p_s,0,999999,-1.77695e-37,1.77695e-37,3mob_s + glu__L_s --> -3.26977488745043e-6*sigma BSU02390-MONOMER_mod_6:pydx5p_s + akg_s + val__L_s
VALTA_FWD_BSU38550-MONOMER_s,0,999999,0,0,akg_s + val__L_s --> 3mob_s + -4.27350427350427e-6*sigma BSU38550-MONOMER_s + glu__L_s
IPPS_FWD_BSU28280-MONOMER_s,0,999999,0,-0,3mob_s + accoa_s + h2o_s --> 3c3hmp_s + -9.60742173454932e-6*sigma BSU28280-MONOMER_s + coa_s + h_s
VALDHr_FWD_BSU24080-MONOMER_s,0,999999,0,0,h2o_s + nad_s + val__L_s --> 3mob_s + -1.25190676006434e-5*sigma BSU24080-MONOMER_s + h_s + nadh_s + nh4_s
MOHMT_FWD_BSU22430-MONOMER_s,0,999999,0,-0,3mob_s + h2o_s + mlthf_s --> 2dhp_s + -4.32708644184951e-6*sigma BSU22430-MONOMER_s + thf_s
VALTA_FWD_BSU02390-MONOMER_mod_6:pydx5p_s,0,999999,0,0,akg_s + val__L_s --> 3mob_s + -3.26977488745043e-6*sigma BSU02390-MONOMER_mod_6:pydx5p_s + glu__L_s


In [4]:
# for r in get_transport_reactions(sporeme,'met__L_s',comps=['c','s']):
#     print(r.id)
#     r.bounds = (0,0)
# solve_me_model(sporeme, max_mu = 0.1, min_mu = .01, using_soplex=False, precision = 1e-3,growth_key = 'sigma')

In [3]:
if sporeme.solution: print(sporeme.solution.x_dict['biomass_dilution_s'])

0.017942047119140624


In [4]:
dct = {}
for m in sporeme.metabolites.query(re.compile('^[a-z]{3}__L_s|^gly_s$')):
    df = flux_based_reactions(sporeme,m.id,only_types=['MetabolicReaction'])
    prod = df[df['met_flux']>0]['met_flux']
    prod = prod.div(prod.sum())
    top = df.sort_values('met_flux').iloc[[-1]]
    print(m)
    dct[m.id] = {}
    if top['met_flux'][0] > 0:
        print(top['reaction'][0])
        dct[m.id]["reaction"] = top['reaction'][0]
        print(top['met_flux'][0])
        dct[m.id]["met_flux"] = top['met_flux'][0]
        print('{} %'.format(str(prod.loc[top.index[0]]*100)))
        dct[m.id]["percentage"] = '{} %'.format(str(prod.loc[top.index[0]]*100))
    else:
        print('No production')
    print('\n')

met__L_s
atp_s + h2o_s + met__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + adp_s + h_s + met__L_s + pi_s
0.00048509737247183564
100.0 %


trp__L_s
h_c + trp__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_s + trp__L_s
0.00015947163451364514
100.0 %


glu__L_s
akg_s + h_s + nadh_s + nh4_s --> -4.27350427350427e-6*sigma BSU22960-MONOMER_s + glu__L_s + h2o_s + nad_s
0.00810987214544222
83.93457550816719 %


ser__L_s
h_c + ser__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_s + ser__L_s
0.0032057865794111124
100.0 %


lys__L_s
h_s + lys__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_c + lys__L_s
0.0018966915533663322
100.0 %


phe__L_s
glu__L_s + phpyr_s --> -4.27350427350427e-6*sigma BSU09570-MONOMER_s + akg_s + phe__L_s
0.0007093732888843219
100.0 %


gly_s
gly_c + h_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + gly_s + h_s
0.0014233087303357351
94.76281121891273 %


leu__L_s
h_c + leu__L_c --> -4.27350427350427e-6*sigma PORE-CPLX_s + h_s + leu__L_s
0.003095526244994408

In [11]:
pd.DataFrame.from_dict(dct).T.to_csv("./Supplementary Files/SupplementaryFile_AA_Production.csv")

# Experimental expression (proteomics)

In [30]:
known_depletions = set(name_to_locus.get(g[0].lower() + g[1:],"") for g in list(pd.read_csv('proteomics/known_depletions.txt')['protein'].values))
name_to_locus = pd.read_csv('./gene_name_dictionary.csv',index_col=1)['locus_id'].to_dict()
locus_to_name = pd.read_csv('./gene_name_dictionary.csv',index_col=0)['name'].to_dict()

In [21]:
model = cobra.io.load_json_model("../../building_data/iYO844_mod.json")

In [27]:
genes = set()
for r in model.reactions:
    if "S_Amino_acids_and_related_molecules" in r.subsystem:
        genes = genes | set(i.id for i in r.genes)

In [77]:
dct = {}
for g in amino_acid_depletions:
    if "protein_"+g not in sporeme.metabolites:
        continue
    dct[g] = {
        "name" : locus_to_name[g],
        "flux" : sporeme.solution.x_dict["translation_"+g+"_s"]
    }

## Comparison of known depletions of amino acid biosynthetic genes and predicted flux

In [103]:
df = pd.DataFrame.from_dict(dct).T.sort_values("flux")
df

,name,flux
BSU28270,leuB,0
BSU11870,metI,0
BSU29440,argH,0
BSU11880,metC,0
BSU11220,argD,0
BSU11200,argJ,0
BSU28310,ilvB,0
BSU15510,pyrAA,0
BSU28250,leuD,0
BSU15850,sdaAB,0


In [104]:
df.to_csv("./Supplementary Files/SupplementaryFile_AA_Genes.csv")

In [120]:
flux_based_reactions(sporeme,"BSU33940-MONOMER_s",growth_symbol="sigma")

,lb,ub,rxn_flux,met_flux,reaction
formation_BSU33940-MONOMER_s,0,1000,1.54057e-07,1.54057e-07,protein_BSU33940_s --> BSU33940-MONOMER_s
GAPD_FWD_BSU33940-MONOMER_s,0,1000,2.77228,-1.05549e-07,g3p_s + nad_s + pi_s --> 13dpg_s + -3.097226360771e-6*sigma BSU33940-MONOMER_s + h_s + nadh_s
GAPD_REV_BSU33940-MONOMER_s,0,1000,0,-0,13dpg_s + h_s + nadh_s --> -3.097226360771e-6*sigma BSU33940-MONOMER_s + g3p_s + nad_s + pi_s
